In [3]:
from IPython.display import display, clear_output
import cv2
from ultralytics import YOLO
import time

KNOWN_WIDTH_INCH = 1.0
KNOWN_DISTANCE_INCH = 12.0
FOCAL_LENGTH = None

# Load your model if not already loaded
# model = YOLO(MODEL_PATH)

cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("❌ Camera not opened")


In [6]:
import cv2
from ultralytics import YOLO
import time
import os

# --- Parameters ---
MODEL_PATH = "trained_yolo_model/ODM-ver5.pt"
KNOWN_WIDTH_INCH = 1.0            # Width of block in inches
ASSUMED_PIXEL_WIDTH = 100.0       # Approx width of block in pixels at ~12 inches
FOCAL_LENGTH = (ASSUMED_PIXEL_WIDTH * 12.0) / KNOWN_WIDTH_INCH

SAVE_DIR = "snapshots"
os.makedirs(SAVE_DIR, exist_ok=True)

# --- Load YOLO model ---
model = YOLO(MODEL_PATH)
print("✅ YOLO model loaded")

# --- Open camera ---
cap = cv2.VideoCapture(0)  # 0 = default camera
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

if not cap.isOpened():
    raise RuntimeError("❌ Could not open camera. Make sure a camera is connected.")

print("Running YOLO + distance estimation headless for 10 seconds...")

start_time_total = time.time()
snapshot_count = 0

try:
    while True:
        ret, frame = cap.read()
        if not ret or frame is None:
            print("❌ Failed to grab frame")
            continue

        # Run YOLO
        start_time = time.time()
        results = model(frame, verbose=False)
        end_time = time.time()

        # Calculate distances
        distances = []
        for box in results[0].boxes.xywh:
            w_pixels = box[2].item()
            distance_inch = (KNOWN_WIDTH_INCH * FOCAL_LENGTH) / w_pixels
            distances.append(distance_inch)

        # Print detected distances
        if distances:
            print(f"Detected {len(distances)} block(s), distances (inches): {[round(d,2) for d in distances]}")

        fps = 1 / (end_time - start_time)
        print(f"FPS: {fps:.2f}")

        # Save snapshot every 5 frames
        snapshot_count += 1
        if snapshot_count % 5 == 0:
            filepath = os.path.join(SAVE_DIR, f"frame_{snapshot_count}.jpg")
            success = cv2.imwrite(filepath, frame)
            if success:
                print(f"✅ Saved {filepath}")
            else:
                print(f"❌ Failed to save {filepath}")

        # Stop after 10 seconds
        if time.time() - start_time_total >= 10:
            print("⏱ 10 seconds elapsed, stopping...")
            break

except KeyboardInterrupt:
    print("Stopped by user")

finally:
    cap.release()
    print("Camera released")
    print("Snapshots saved in folder:", os.path.abspath(SAVE_DIR))


✅ YOLO model loaded
Running YOLO + distance estimation headless for 10 seconds...
Detected 1 block(s), distances (inches): [14.52]
FPS: 0.37
Detected 1 block(s), distances (inches): [14.58]
FPS: 0.50
Detected 1 block(s), distances (inches): [14.59]
FPS: 0.47
Detected 1 block(s), distances (inches): [14.46]
FPS: 0.50
Detected 1 block(s), distances (inches): [14.53]
FPS: 0.48
✅ Saved snapshots/frame_5.jpg
⏱ 10 seconds elapsed, stopping...
Camera released
Snapshots saved in folder: /home/clopezgarcia2/Desktop/roboforge/robofore-vision/snapshots


✅ YOLO model loaded
Running YOLO + distance & position estimation headless for 10 seconds...
No blocks detected in this frame.
FPS: 0.48

No blocks detected in this frame.
FPS: 0.54

No blocks detected in this frame.
FPS: 0.55

No blocks detected in this frame.
FPS: 0.57

Block 1: Distance = 12.97 in, Position = (46, 61) px
Block 2: Distance = 14.79 in, Position = (473, 170) px
Mapped position: x=-27.4 in, y=13.0 in
Mapped position: x=15.3 in, y=14.8 in
FPS: 0.58

✅ Saved snapshots/frame_5.jpg
Block 1: Distance = 12.95 in, Position = (46, 61) px
Mapped position: x=-27.4 in, y=12.9 in
FPS: 0.47

⏱ 10 seconds elapsed, stopping...
Camera released
Snapshots saved in folder: /home/clopezgarcia2/Desktop/roboforge/robofore-vision/snapshots


In [13]:
import cv2
from ultralytics import YOLO
import time
import os

# --- Parameters ---
MODEL_PATH = "trained_yolo_model/ODM-ver5.pt"
KNOWN_WIDTH_INCH = 1.0            # Width of block in inches
ASSUMED_PIXEL_WIDTH = 100.0       # Approx width of block in pixels at ~12 inches
FOCAL_LENGTH = (ASSUMED_PIXEL_WIDTH * 12.0) / KNOWN_WIDTH_INCH

SAVE_DIR = "snapshots"
os.makedirs(SAVE_DIR, exist_ok=True)

# --- Load YOLO model ---
model = YOLO(MODEL_PATH)
print("✅ YOLO model loaded")

# --- Open camera ---
cap = cv2.VideoCapture(0)  # 0 = default camera
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

if not cap.isOpened():
    raise RuntimeError("❌ Could not open camera. Make sure a camera is connected.")

print("Running YOLO + distance & position estimation headless for 10 seconds...")

start_time_total = time.time()
snapshot_count = 0

try:
    while True:
        ret, frame = cap.read()
        if not ret or frame is None:
            print("❌ Failed to grab frame")
            continue

        # --- YOLO inference ---
        start_time = time.time()
        results = model(frame, verbose=False)
        end_time = time.time()

        # --- Calculate distances and positions ---
        block_info = []  # (distance_inch, center_x_px, center_y_px)
        for box in results[0].boxes.xywh:
            cx, cy, w_pixels, h_pixels = box
            distance_inch = (KNOWN_WIDTH_INCH * FOCAL_LENGTH) / w_pixels.item()
            block_info.append((round(distance_inch, 2), int(cx.item()), int(cy.item())))

        # --- Print detected blocks ---
        if block_info:
            for i, (dist, x, y) in enumerate(block_info):
                print(f"Block {i+1}: Distance = {dist} in, Position = ({x}, {y}) px")

            # --- Optional: simple 2D map estimation ---
            map_scale = 0.1  # 1 pixel = 0.1 inch
            for dist, x, y in block_info:
                map_x = (x - 320) * map_scale  # horizontal offset from center
                map_y = dist                   # forward distance
                print(f"Mapped position: x={map_x:.1f} in, y={map_y:.1f} in")
        else:
            print("No blocks detected in this frame.")

        # --- FPS ---
        fps = 1 / (end_time - start_time)
        print(f"FPS: {fps:.2f}\n")

        # --- Annotate frame ---
        annotated_frame = frame.copy()
        for i, (dist, cx, cy) in enumerate(block_info):
            # Convert xywh to top-left and bottom-right
            box = results[0].boxes.xywh[i]
            x1 = int(box[0].item() - box[2].item()/2)
            y1 = int(box[1].item() - box[3].item()/2)
            x2 = int(box[0].item() + box[2].item()/2)
            y2 = int(box[1].item() + box[3].item()/2)
            cv2.rectangle(annotated_frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(annotated_frame, f"{dist:.2f} in", (x1, y1-10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
            cv2.circle(annotated_frame, (cx, cy), 3, (0, 0, 255), -1)

        # --- Save annotated snapshot every 5 frames ---
        snapshot_count += 1
        if snapshot_count % 5 == 0:
            filepath = os.path.join(SAVE_DIR, f"frame_{snapshot_count}.jpg")
            success = cv2.imwrite(filepath, annotated_frame)
            if success:
                print(f"✅ Saved annotated {filepath}")
            else:
                print(f"❌ Failed to save {filepath}")

        # --- Stop after 10 seconds ---
        if time.time() - start_time_total >= 10:
            print("⏱ 10 seconds elapsed, stopping...")
            break

except KeyboardInterrupt:
    print("Stopped by user")

finally:
    cap.release()
    print("Camera released")
    print("Snapshots saved in folder:", os.path.abspath(SAVE_DIR))


✅ YOLO model loaded
Running YOLO + distance & position estimation headless for 10 seconds...
Block 1: Distance = 13.03 in, Position = (46, 62) px
Mapped position: x=-27.4 in, y=13.0 in
FPS: 0.48

Block 1: Distance = 12.95 in, Position = (46, 62) px
Block 2: Distance = 14.85 in, Position = (473, 169) px
Mapped position: x=-27.4 in, y=12.9 in
Mapped position: x=15.3 in, y=14.8 in
FPS: 0.60

Block 1: Distance = 12.98 in, Position = (46, 61) px
Block 2: Distance = 14.73 in, Position = (472, 169) px
Mapped position: x=-27.4 in, y=13.0 in
Mapped position: x=15.2 in, y=14.7 in
FPS: 0.56

Block 1: Distance = 14.72 in, Position = (472, 169) px
Block 2: Distance = 13.02 in, Position = (46, 60) px
Mapped position: x=15.2 in, y=14.7 in
Mapped position: x=-27.4 in, y=13.0 in
FPS: 0.60

Block 1: Distance = 12.92 in, Position = (46, 61) px
Block 2: Distance = 14.69 in, Position = (472, 169) px
Mapped position: x=-27.4 in, y=12.9 in
Mapped position: x=15.2 in, y=14.7 in
FPS: 0.59

✅ Saved annotated sn